# Deep Learning & Applied AI

We recommend going through the notebook using Google Colaboratory.

# Tutorial 5: Autograd and Modules

In this tutorial, we will cover:

- Autograd, back-propagation
- Modules, `torch.nn`

Our info:

- Dr. Luca Moschella (moschella@di.uniroma1.it)
- Dr. Marco Fumero (fumero@di.uniroma1.it)

Course:

- Website and notebooks will be available at [DLAI-s2-2023](https://github.com/erodola/DLAI-s2-2023/)

## Import dependencies (run the following cells)

In [1]:
# @title import dependencies

from typing import Mapping, Union, Optional

import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import plotly.graph_objects as go

from torchvision import datasets, transforms
from tqdm.notebook import tqdm

In [2]:
# @title reproducibility stuff

import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)

torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False

In [3]:
# @title utility functions

from typing import Callable, Union, Sequence
import math

def peaks(meshgrid: torch.Tensor) -> torch.Tensor:
  """ 
  "Peaks" function that has multiple local minima.

  :params meshgrid: tensor of shape [..., 2], the (x, y) coordinates
  """
  meshgrid = torch.as_tensor(meshgrid, dtype=torch.float)
  xx = meshgrid[..., 0]
  yy = meshgrid[..., 1]
  return (0.25 * (3*(1-xx)**2*torch.exp(-xx**2 - (yy+1)**2) - 
                  10*(xx/5 - xx**3 - yy**5)*torch.exp(-xx**2-yy**2) - 
                  1/3*torch.exp(-(xx+1)**2 - yy**2)))


def rastrigin(meshgrid: torch.Tensor, shift: int = 0) -> torch.Tensor:
  """ 
  "Rastrigin" function with `A = 3` 
  https://en.wikipedia.org/wiki/Rastrigin_function

  :params meshgrid: tensor of shape [..., 2], the (x, y) coordinates
  """
  meshgrid = torch.as_tensor(meshgrid, dtype=torch.float)
  xx = meshgrid[..., 0]
  yy = meshgrid[..., 1]
  A = 3
  return A * 2 + (((xx - shift) ** 2 - A * torch.cos(2 * torch.tensor(math.pi, dtype=torch.float, device=xx.device) * xx))
                  +
                  ((yy - shift) ** 2 - A * torch.cos(2 * torch.tensor(math.pi, dtype=torch.float, device=xx.device) * yy)))


def rosenbrock(meshgrid: torch.Tensor) -> torch.Tensor:
  """
  "Rosenbrock" function
  https://en.wikipedia.org/wiki/Rosenbrock_function

  It has a global minimum at $(x , y) = (a, a^2) = (1, 1)$

  :params meshgrid: tensor of shape [..., 2], the (x, y) coordinates
  """
  meshgrid = torch.as_tensor(meshgrid, dtype=torch.float)
  xx = meshgrid[..., 0]
  yy = meshgrid[..., 1]
  
  a = 1
  b = 100
  return (a - xx) ** 2 + b * (yy - xx**2)**2


def simple_fn(meshgrid: torch.Tensor) -> torch.Tensor:
  """
  :params meshgrid: tensor of shape [..., 2], the (x, y) coordinates
  """
  meshgrid = torch.as_tensor(meshgrid, dtype=torch.float)
  xx = meshgrid[..., 0]
  yy = meshgrid[..., 1]
  
  output = -1/(1 + xx**2 + yy**2) 

  return output

def simple_fn2(meshgrid: torch.Tensor) -> torch.Tensor:
  """
  :params meshgrid: tensor of shape [..., 2], the (x, y) coordinates
  """
  meshgrid = torch.as_tensor(meshgrid, dtype=torch.float)
  xx = meshgrid[..., 0]
  yy = meshgrid[..., 1]
  
  output = (1 + xx**2 + yy**2) ** (1/2)

  return output

In [4]:
# @title utility plot functions
import plotly.express as px

def plot_landscape(
    fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    resolution: int = 100,
    lim: int = 3,
    height: int = 900,
    landscape_opacity: float = 1.0,
    title: Optional[str] = None,
    autoshow: bool = False,
    **kwargs
) -> go.Figure:
    """ Plot the landscape defined by the function `fn`.

  Creates a domain grid $x,y \in R^2$ with $x \in [-lim, lim]$ and
  $y \in [-lim, lim]. The number of points in this grid is resolution**2.
  """
    xx = torch.linspace(-lim, lim, resolution)
    yy = torch.linspace(-lim, lim, resolution)

    yy = yy.repeat(yy.shape[0], 1)
    xx = xx.unsqueeze(-1).repeat(1, xx.shape[0])
    meshgrid = torch.stack((xx, yy), dim=-1)
    zz = fn(meshgrid, **kwargs)

    xx = xx.cpu().detach()
    yy = yy.cpu().detach()
    zz = zz.cpu().detach()

    fig = go.Figure(data=[go.Surface(z=zz, x=xx, y=yy, opacity=landscape_opacity, 
                                     cmid=0,
                                     colorscale='Viridis')])
    fig.update_traces(
        contours_z=dict(
            show=True, usecolormap=True, highlightcolor="lightgray", project_z=True
        )   
    )
    fig.update_layout(
        height=height,
    )

    if autoshow:
      fig.show()
    return fig


def plot_points_over_landscape(
    fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    points: (float, float) = None,
    resolution: int = 100,
    lim: int = 3,
    landscape_opacity: float = 1.0,
    height: int = 900,
    title: Optional[str] = None,
    autoshow: bool = False,
) -> go.Figure:
    """ Plot a point over the landascape defined by the cunction `fn`

    :param fn: an universal function $R^2 -> R$
    :param points: tensor of shape [..., 3]
    :param title: the title of the plots, if None defaults to  the fn name
    :param autoshow: if True, calls fig.show() before returning the figure

    :retuns: the figure that contains the plot
    """
    points = torch.as_tensor(points)
    fig = plot_landscape(
        fn,
        resolution=resolution,
        lim=lim,
        height=height,
        landscape_opacity=landscape_opacity,
        title=title
    )

    # Create starting path
    x_points = points[..., 0] 
    y_points = points[..., 1]
    z_points = points[..., 2]

    for point in points:
      fig.add_trace(
          go.Scatter3d(
              visible=True,
              showlegend=False,
              mode="markers",
              marker=dict(size=6, color="darkred", symbol="circle"),
              x=x_points,
              y=y_points,
              z=z_points,
          )
      )

    if autoshow:
        fig.show()

    return fig

# Intro


# Autograd: automatic differentiation



Let's begin.

---

We have seen already many PyTorch features, from all the linear algebra tools to the useful ``Dataset`` and ``Optimizer`` classes.

And yet, with little effort we could have used the Numpy or Sklearn libraries instead of Pytorch to accomplish the feats of the last notebooks.

Today we will go through the main raison d'être of Pytorch; the ``autograd`` package.

### Differentiable programming: a new programming paradigm


>Differentiable programming is a programming paradigm in which the programs can be differentiated throughout, usually via **automatic differentiation**. This allows for gradient based optimization of parameters in the program, often via gradient descent. Differentiable programming has found use in a wide variety of areas, particularly scientific computing and artificial intelligence. (*Wikipedia [entry](https://en.wikipedia.org/wiki/Differentiable_programming) for Differentiable programming*)


The ``autograd`` package of PyTorch is here to provide you this *automatic differentiation* for all operations
on Tensors. 

It follows a *define-by-run* philosophy, which means that the backpropagation is
defined by how your code is run, and that **every single iteration can be
different**.  


As opposed to the *static-computational-graphs* philosophy of doing automatic differentiation (e.g. the one used in TensorFlow 1.0)  which can not be changed between different iterations. (*but allow a higher degree of graph-based optimization*)

Both these approaches are based on the *reverse automatic differentation* way of computing derivatives.

As seen in lecture, it scales to **high dimensional data** and **very complex computational graphs**, differently from the forward approach.

In the context of Neural Networks we refer to the reverse automatic differentation as *backpropagation*.

## Basics

Let's start by defining a tensor $x$ useful in some computation $f$, like $f(x) = x^2 + x^3$.

Suppose we would like to calculate its rate of change at the point $x=42$, i.e the derivative:
$$\frac{\partial f}{\partial x}\Bigr\rvert_{x=42}$$

PyTorch does it through the *reverse mode automatic differentiation*, composed of a forward and backward pass, as seen in lecture.

So we first need to define the forward pass.

As said before, PyTorch defines the forward pass **as you run** it.

In [5]:
x = torch.tensor(42., requires_grad=True)  # we want the gradient respect to this variable!
x2 = x ** 2
x3 = x ** 3
f = x2 + x3

Now the backward pass. 

Finally we can appreciate the *automatic* part of the differentation, to perform the backward pass you just need to call the `backward` method from your output $f$.  

In [6]:
f.backward()

Now you have $\frac{\partial f}{\partial x}\Bigr\rvert_{x=42}$ in the `grad` of $x$

In [7]:
x.grad

tensor(5376.)

In [8]:
2 * 42 + 3 * 42**2  # Yep, it's correct.

5376

Now you know the basics. And this is enough for training very standard models on PyTorch.

Nevertheless, the design principles behind the PyTorch `autograd` package are not always as straight-forward. For instance, what do you think will happen executing `backward` a second time?





In [9]:
# f.backward()  # try!


To fully understand the world of the `Autograd` package we must go deeply down the rabbit hole.

You do not need to get at the first pass everything we are going to mention from now on. There are explanations of advanced concepts and some PyTorch internals, which are usually not needed but can be useful (e.g. in debugging or complex implementations).

Every time we will cross a complexity level, we are going to signal it with 🔵 🔴:


- Blue pill, the story ends and you can safely skip to the *Torch.nn package* section.

- Red pill, you stay in wonderland and we will show you how deep the rabbit hole goes.


### `Autograd` aggressive buffer freeing

#### The second backward
So, what was the problem with the second backward? 

When we have computed the first `backward`, the intermediate variables needed to compute $f$ and its gradient have been freed by default to save memory. So PyTorch do not have the necessary information to do backward from $f$ a second time.

`Autograd` has an aggressive buffer freeing policy to be very memory efficient!



If you want to keep intermediary results needed to be able to recompute `backward`, you can use `.backward(retain_graph=True)`.

Let's redo from scratch the previous computation

In [10]:
x = torch.tensor(42., requires_grad=True) 
x2 = x ** 2
x3 = x ** 3
f = x2 + x3
f.backward(retain_graph=True)
f.backward()

So we did backward two times. Let's now check newly the gradient of $x$

In [11]:
x.grad

tensor(10752.)

It's doubled! `Autograd` accumulates into `grad` attribute. This means that multiple calls to **`backward` will sum up previously computed gradients** if they are not zeroed.

#### Intermediate gradients are not kept by default
Intermediate gradients are another victim of PyTorch aggressive buffer freeing policy.

We do not have access to the gradient with respect to $x_2$, even if we actually computed it to calculate the one respect to $x$.

In [12]:
x2.requires_grad  # we *require* the grad respect to x2 to compute the one respect to x...

True

In [13]:
x2.grad is None  # ...but we have asked pytorch only the gradient respect to x, so the one respect to x2 is not maintained in memory

<ipython-input-13-53c7818ab4c9>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:480.)
  x2.grad is None  # ...but we have asked pytorch only the gradient respect to x, so the one respect to x2 is not maintained in memory


True

### Sick of being tracked? 🍪

You can call `detach()` to detach a tensor from the computational graph and prevent future computation from being tracked. You can also wrap the code block in a scope `with torch.no_grad():`

In [15]:
x = torch.tensor(42., requires_grad=True) 
x2 = x ** 2
x2sig = x2
print(x2sig.requires_grad)
x2nog = x2.detach()
print(x2nog.requires_grad)

True
False


In [16]:
x = torch.tensor(42., requires_grad=True) 
x2 = x ** 2
print(x2.requires_grad)
with torch.no_grad():
    x2nog = x ** 2
    x3nog = (x2 + 7) ** 3
    print(x2nog.requires_grad)    
    print(x3nog.requires_grad)

True
False
False


You won't be able to backpropagate trough a detached tensor:

In [17]:
try:
  x2nog.sum().backward()
except Exception as e:
  print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [18]:
# it works if it has not been detached:
x2sig.sum().backward()

### Tensors 🔵 🔴



``torch.Tensor`` is the central class of the `autograd` package. 


In order to understand in detail how autograd works, it is necessary to dissect some of the most relevant attributes of the Tensors:

---

- **`data`**: 

It is the data the tensor is holding. Usually you do not need to access directly this attribute.

In [19]:
t = torch.rand(4, 4)
t.data

tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411, 0.4294]])


---

- **`requires_grad`**: 

Is `True` if gradients need to be computed for this Tensor, `False `otherwise.**The fact that gradients need to be computed for a Tensor do not mean that the `grad` attribute will be populated**, the tensor must also be a leaf! See `is_leaf` for more details. Every tensor produced by tensors that require grad, 
in turn requires grad.




In [20]:
t.requires_grad

False

In [21]:
t2 = torch.rand(4, 4, requires_grad=True)
t2.requires_grad

True

---

- **`grad`**: 

This attribute is `None` by default and becomes a Tensor the first time a call to `backward()` computes gradients for `t`. 
The attribute will then contain the gradients computed and future calls to `backward()` will accumulate (add) gradients into it. Only the leaves nodes of the computational graph with `requires_grad=True` will have the `grad` attribute populated.



In [22]:
print(t.grad, t2.grad)

None None



---

- **`grad_fn`**: 

The backward function to use to compute the gradient, it depends of which operations has been performed in the forward pass!

In [23]:
(t + t2)

tensor([[1.7677, 1.4889, 0.6494, 1.5868],
        [0.6601, 1.0423, 0.5535, 1.6253],
        [1.0461, 0.4027, 1.2934, 0.7929],
        [1.4166, 0.5739, 1.6926, 0.5047]], grad_fn=<AddBackward0>)

In [24]:
(t + t)  # t does not have `requires_grad=True`, thus grad_fn is not needed

tensor([[1.7645, 1.8300, 0.7657, 1.9186],
        [0.7809, 1.2018, 0.5131, 1.5873],
        [1.8815, 0.2664, 1.8692, 1.1872],
        [1.7388, 1.1354, 1.4822, 0.8588]])


---

- **`is_leaf`**: a boolean. 

**Only *leaf* Tensors with `requires_grad=True` will have their `grad` populated during a call to `backward()`**. To get `grad` populated for non-leaf Tensors, you can use `retain_grad()`. 
Keep in mind that:
  - All Tensors that have `requires_grad=False` will be leaf Tensors by convention.
  - For Tensors that have `requires_grad=True`, they will be leaf Tensors if their `grad_fn` is `None`. This means that they are not the result of an operation of tracked tensors, thus they were created directly by the user.

In [25]:
a = torch.rand(10, requires_grad=True)
a.is_leaf, a.requires_grad

(True, True)

In [26]:
a = torch.rand(10, requires_grad=True).cuda()
a.is_leaf, a.requires_grad  # was created by the operation that cast a cpu Tensor into a cuda Tensor

(False, True)

In [27]:
a = torch.rand(10, requires_grad=True, device="cuda")
a.is_leaf, a.requires_grad  # requires grad, directly created by the user

(True, True)

In [28]:
a = torch.rand(10, requires_grad=True) + 2
a.is_leaf, a.requires_grad  # was created by the addition operation

(False, True)

In [29]:
a = torch.rand(10).cuda()
a.is_leaf, a.requires_grad  # does not require gradients thus it is a leaf by convention

(True, False)

In [30]:
a = torch.rand(10).cuda().requires_grad_()  # defaults to requires_grad=True
a.is_leaf, a.requires_grad  # requires gradients and has `grad_fn=None`

(True, True)

In [31]:
a = torch.rand(10, requires_grad=True, device="cuda")
b = a + 2                          # non leaf, since requires grad and it is produced by an op
print(b.is_leaf, b.requires_grad)  
c = b.detach()                     # leaf, it has been detached and now has requires_grad=False
print(c.is_leaf, c.requires_grad)  

False True
True False


---

- **`backward()`**: 

Computes the gradient of current tensor w.r.t. computational graph leaves.

The graph is differentiated using the chain rule. If the tensor is non-scalar (i.e. its data has more than one element) and requires gradient, the function additionally requires specifying gradients. It should be a tensor of matching type and location, that contains the external gradient of the differentiated external (scalar) function w.r.t. current tensor. 

> To be more clear, if you specify the gradients you should think as the current tensor being an intermediate tensor of a bigger unknown computational graph, where another unknown scalar is being differentiated. E.g. if for some strange reason you want to backpropagate between PyTorch *and* Tensorflow code, you will need to explicitly provide the gradients to "connect" the two computational graphs. *(...don't do it, it is easier and safer to just port the code)*

This function accumulates gradients in the leaves - you might need to zero them before calling it.

Remember, the backward graph is made dynamically during the forward pass. Backward function only calculates the gradient using this graph and stores them in leaf nodes.

> ### Leaves recap
>
> Let's recap the answer to the following question:
>
> *Which are the nodes that will have the `.grad` attribute populated?*
>
> Intuitively, you can detect those nodes in this way.
>
> Given a computational graph:
>
> ![](https://raw.githubusercontent.com/erodola/DLAI-s2-2021/main/labs/05/pics/leaves.svg)
>
> 1. Take the subgraph of nodes with `requires_grad=True` *(green and blue nodes)*
> 2. Take the leaves of this subgraph *(green nodes)*
>
> The nodes selected with this procedure *(green nodes)* will have their `.grad` attribute populated.

### Gradients

Let's see some examples of gradients computations, to recap what we just saw:

> If you want to compute the derivatives, you can call ``backward()`` on
a ``Tensor``:
> 
> - If ``Tensor`` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to ``backward()``. 
> 
> - If it has more elements, you need to specify a ``gradient`` argument that is a tensor of matching shape. 
> The ``gradient`` argument is only a way to inject external gradients: it considers the last non-scalar tensor from which you call `backward` as an intermediate tensor in the computational graph (i.e. it assumes there is a loss function external to PyTorch).

Create a tensor and set ``requires_grad=True`` to track computation with it:

In [32]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

Do an operation on the tensor:


In [33]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

``y`` was created as a result of a tracked operation, so it has a ``grad_fn``:



In [34]:
y.grad_fn

Do more operations on y:

In [35]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [36]:
out.backward()

With this operation we computed $\frac{\partial \, \text{out}}{\partial \, x}$:


In [37]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

Let's see why `x.grad` is a `2x2` tensor full of fours. 

The output is defined as:

$$ o = \frac{1}{4} \sum_i 3(x_i + 2)^2 \: \text{ with } x_i = 1 \, \forall i$$

We need to compute $\frac{\partial o}{\partial x_i} \; \forall  i$:

$$
\frac{\partial o}{\partial x_i} 
= \frac{3 \times 2}{4} (x_i + 2)
= \frac{3}{2} (x_i + 2)
$$

*(The gradient for every $x_j$ with $j \neq i$ is zero)*


Thus, $\frac{\partial o}{\partial x_i}|_{x_i=1} = \frac{9}{2} = 4.5$  

##### **EXERCISE**
> Understanding if a tensor is a leaf or not is suprisingly tricky, but it is very important to be able to distinguish leaf tensors: **only leaves with `requires_grad=True` tensors will have the grad attribute populated**.
>
> Consider the two following scenarios and try to understand if `a.grad` and/or `b.grad` will be populated.
>
> **Scenario 1**
>
> ```python
> a = torch.randn(2, 2, requires_grad=True) 
> b = a ** 2                                
> b.requires_grad_(True)                    
> b.sum().backward()                        
> ```
> - [ ] `a.grad` is populated (it is not `None`)
> - [ ] `b.grad` is populated (it is not `None`)
>
>
> **Scenario 2**
>
> ```python
> a = torch.randn(2, 2, requires_grad=False) 
> b = a ** 2                                
> b.requires_grad_(True)                    
> b.sum().backward()                        
> ```
> - [ ] `a.grad` is populated (it is not `None`)
> - [ ] `b.grad` is populated (it is not `None`)

In [41]:
# @title Solution 👀

if False:  # Change to true to enable the prints
  # 1)
  a = torch.randn(2, 2, requires_grad=True)  # leaf tensor that requires grad

  b = a ** 2                                 # non leaf tensor: requires grad and produced by an op
  b.requires_grad_(True)                     # it already requires a grad!

  print(f'a.is_leaf: {a.is_leaf} \t a.requires_grad: {a.requires_grad}  \t a.grad_fn: {a.grad_fn}') 
  print(f'b.is_leaf: {b.is_leaf} \t b.requires_grad: {b.requires_grad}  \t b.grad_fn: {b.grad_fn}') 

  b.sum().backward()                         # just a sample backprop

  print("\nGradients:")
  print(f'a.grad: {a.grad}')                 # a is a leaf, thus it will have .grad
  print(f'b.grad: {b.grad}')                 # b is not a leaf, thus it will not have .grad

  print('\n\n---\n\n')

  # 2)
  a = torch.randn(2, 2, requires_grad=False) # leaf tensor that does not requires grad

  b = a ** 2                                 # leaf tensor, because not requires grad
  b.requires_grad_(True)                     # now it requires a grad and has grad_fn=None! It is a leaf

  print(f'a.is_leaf: {a.is_leaf} \t a.requires_grad: {a.requires_grad}  \t a.grad_fn: {a.grad_fn}') 
  print(f'b.is_leaf: {b.is_leaf} \t b.requires_grad: {b.requires_grad}  \t b.grad_fn: {b.grad_fn}') 

  b.sum().backward()                         # just a sample backprop

  print("\nGradients:")
  print(f'a.grad: {a.grad}')                 # a is not a leaf, thus it will not have .grad
  print(f'b.grad: {b.grad}')                 # b is a leaf, thus it will have .grad

  print('\n\n---\n\n')           

##### **EXERCISE**
>
> Consider the following expression:
>
> $$ z = \frac{\sqrt{x^2 +1} - \sqrt{y - 1}}{\sqrt{x^2 + y^2}} + \sqrt{y - 1} $$
>
> Compute the gradients $\frac{\partial z}{\partial x}$, $\frac{\partial z}{\partial y}$, $\frac{\partial z}{\partial \sqrt{x^2 +1}}$ and $\frac{\partial z}{\partial \sqrt{y-1}}$ at $x=2$, $y=10$

In [42]:
# Expected results, respectively:
# x.grad: 0.08914636820554733
# y.grad: 0.15752650797367096
# x3.grad: 0.0980580672621727
# y2.grad: 0.9019419550895691

### Autograd Mechanics 🔵 🔴



#### Custom `Function`

There’s one more class which is very important for autograd
implementation; a ``Function``. 

``Tensor`` and ``Function`` are interconnected and build up an **acyclic computational graph**, that encodes a complete history of computation. **Each tensor** has a ``grad_fn`` attribute that references a ``Function`` that has created the ``Tensor`` (except for leaf Tensors created by the user - their
``grad_fn is None``).

All mathematical operations in PyTorch are implemented with this `torch.nn.Autograd.Function` class. 

Adding operations to autograd requires implementing a new `Function` subclass for each operation. Recall that `Function`s are what autograd uses to compute the results and gradients, and encode the operation history. Every new function requires you to implement 2 methods:

- `forward()`: the code that performs the operation. It can take as many arguments as you want, with some of them being optional, if you specify the default values. All kinds of Python objects are accepted here. `Tensor` arguments that track history (i.e., with `requires_grad=True`) will be converted to ones that don’t track history before the call, and their use will be registered in the graph. Note that this logic won’t traverse lists/dicts/any other data structures and will only consider `Tensor` s that are direct arguments to the call. You can return either a single `Tensor` output, or a tuple of `Tensor` s if there are multiple outputs. Also, please refer to the docs of `Function` to find descriptions of useful methods that can be called only from `forward()`.

- `backward()`: gradient formula. It will be given as many `Tensor` arguments as there were outputs in the `forward()`, with each of them representing gradient w.r.t. that output. It should return as many `Tensor` s as there were inputs in the `forward()`, with each of them containing the gradient w.r.t. its corresponding input. If your inputs didn’t require gradient (`needs_input_grad`, in the `ctx` argument, is a tuple of booleans indicating whether each input needs gradient computation), or were non-Tensor objects, you can return `None`. Also, if you have optional arguments to `forward()` you can return more gradients than there were inputs, as long as they’re all `None`.




In [43]:
class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, x):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(x)

        # Do some operation, even external to PyTorch
        x_device = x.device
        x_dtype = x.dtype
        xnumpy = x.cpu().detach().numpy()
        xnumpy = xnumpy.clip(min=0)  # numpy op: it is not differentiable!
        x = torch.tensor(xnumpy, dtype=x_dtype, device=x_device)
        
        return x

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

myrelu = MyReLU.apply  # alias the `.apply` method to ease its usage

In [44]:
x = torch.rand(50, requires_grad=True) 

In [45]:
out = x - .5
out = myrelu(out)  
print(out)  # grad_fn=<MyReLUBackward>
out.sum().backward()
x.grad      # Negative numbers get zeroed, and their grad is zero

tensor([0.0000, 0.0000, 0.0000, 0.1249, 0.0000, 0.0000, 0.0117, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.4998, 0.0944, 0.1541, 0.0000, 0.0000, 0.0000,
        0.0782, 0.0000, 0.0000, 0.0000, 0.0014, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.4192, 0.0000, 0.4302, 0.1558, 0.0000, 0.3460,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1431, 0.0000, 0.1947, 0.0000, 0.3712,
        0.0000, 0.0000, 0.1044, 0.2581, 0.4037], grad_fn=<MyReLUBackward>)


tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.])

In [46]:
x.grad.zero_()  # usually you should not use this method

# -> Let's check our implementation against torch.relu
out = x - .5
out = torch.relu(out)  
print(out)  # grad_fn=<MyReLUBackward>
out.sum().backward()
x.grad      # Negative numbers get zeroed, and their grad is zero

tensor([0.0000, 0.0000, 0.0000, 0.1249, 0.0000, 0.0000, 0.0117, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.4998, 0.0944, 0.1541, 0.0000, 0.0000, 0.0000,
        0.0782, 0.0000, 0.0000, 0.0000, 0.0014, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.4192, 0.0000, 0.4302, 0.1558, 0.0000, 0.3460,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1431, 0.0000, 0.1947, 0.0000, 0.3712,
        0.0000, 0.0000, 0.1044, 0.2581, 0.4037], grad_fn=<ReluBackward0>)


tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.])

#### Excluding subgraphs from backward

The `requires_grad` flag allows for fine grained exclusion of subgraphs from gradient computation and can increase efficiency.

If there’s a single input to an operation that requires gradient, its output will also require gradient. Conversely, only if all inputs don’t require gradient, the output also won’t require it. **Backward computation is never performed in the subgraphs, where all Tensors didn’t require gradients**.

In [47]:
x = torch.randn(5, 5)  # requires_grad=False by default
y = torch.randn(5, 5)  # requires_grad=False by default
z = torch.randn((5, 5), requires_grad=True)
a = x + y
a.requires_grad

False

In [48]:
b = a + z
b.requires_grad

True

This is especially useful when you want to **freeze part of your model**, or you know in advance that you’re not going to use gradients w.r.t. some parameters. For example if you want to **finetune** the last layer of a pretrained CNN, it’s enough to switch the requires_grad flags in the frozen base, and no intermediate buffers will be saved, until the computation gets to the last layer, where the affine transform will use weights that require gradient, and the output of the network will also require them.

In [49]:
import torchvision
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [50]:
random_out = model(torch.rand(2, 3, 224, 224))  # Compute something
random_out.sum().backward()                     # Compute some gradients

In [51]:
# Return all grads different than all zeros
grads = list(x.grad for x in model.parameters() if x.grad.bool().any())
len(grads)

62

In [52]:
model.zero_grad()

In [53]:
# Freeze the pretrained model
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully-connected layer
# Parameters of constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 100)

# Optimize only the last layer
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)  # model.fc.parameters() explicitly tells which params to optimize

In [54]:
random_out = model(torch.rand(2, 3, 224, 224))  # Compute something
random_out.sum().backward()                     # Compute some gradients

In [55]:
# Return all grads different than all zeros
grads = list(x.grad for x in model.parameters() if x.grad.bool().any())
len(grads)

2

In [56]:
grads

[tensor([[1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872],
         [1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872],
         [1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872],
         ...,
         [1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872],
         [1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872],
         [1.7761, 1.8444, 1.7347,  ..., 1.7706, 2.0340, 1.9872]]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])]

#### How autograd encodes the history

Autograd is a reverse automatic differentiation system. Conceptually, autograd records a graph recording all of the operations that created the data as you execute operations, giving you a directed acyclic graph whose leaves are the input tensors and roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

Internally, autograd represents this graph as a graph of `Function` objects (really expressions), which can be `apply()` ed to compute the result of evaluating the graph. When computing the forwards pass, autograd simultaneously performs the requested computations and builds up a graph representing the function that computes the gradient (the `grad_fn` attribute of each `torch.Tensor` is an entry point into this graph). When the forwards pass is completed, we evaluate this graph in the backwards pass to compute the gradients.

An important thing to note is that the graph is recreated from scratch at every iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change the overall shape and size of the graph at every iteration. You don’t have to encode all possible paths before you launch the training - what you run is what you differentiate.

In [57]:
x = torch.arange(5, dtype=torch.float, requires_grad=True)
y = x - x
v = y ** 2
z = v + 2
z.grad_fn

In [58]:
z.grad_fn.next_functions  # The computational graph is encoded in the grad_fn attributes

((<PowBackward0 at 0x7fe18c6d7a60>, 0), (None, 0))

In [59]:
z.grad_fn.next_functions[0][0].next_functions  # If we want, we can manually traverse it

((<SubBackward0 at 0x7fe18243e130>, 0),)

In [60]:
z.grad_fn.next_functions[0][0].next_functions[0][0].next_functions  # Leaf tensors have private functions to accumulate grads

((<AccumulateGrad at 0x7fe18243e1f0>, 0),
 (<AccumulateGrad at 0x7fe18243e1f0>, 0))

In [61]:
# After the leaf tensors, we don't compute anything! 
# Even if there are other leaf tensor in the computational graph behind those (that do not require a grad)
z.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].next_functions 


()

#### In-place operations with autograd

Now, it should be clear why all the operations **should not be in-place**: we may overwrite some value that we need, breaking the backpropagation mechanism

Supporting in-place operations in autograd is a hard matter, and we discourage their use in most cases. Autograd’s aggressive buffer freeing and reuse makes it very efficient and there are very few occasions when in-place operations actually lower memory usage by any significant amount. Unless you’re operating under heavy memory pressure, you might never need to use them.

There are two main reasons that limit the applicability of in-place operations:

- In-place operations can potentially **overwrite values required to compute gradients**.

- Every in-place operation actually requires the implementation to rewrite the computational graph. **Out-of-place versions simply allocate new objects and keep references** to the old graph, while in-place operations, require changing the creator of all inputs to the `Function` representing this operation. This can be tricky, especially if there are many `Tensors` that reference the same storage (e.g. created by indexing or transposing), and in-place functions will actually raise an error if the storage of modified inputs is referenced by any other `Tensor`.


#### In-place correctness checks

Every tensor keeps a version counter, that is incremented every time it is marked dirty by in-place operations. When a Function saves references of any tensors for its backward pass, a version counter of their containing Tensor is saved as well. Once you access `self.saved_tensors`, the version is checked. If it is greater than the saved value an error is raised. This ensures that if you’re using in-place functions and not seeing any errors, you can be sure that the computed gradients are correct.

In [62]:
x = torch.rand(10, requires_grad=True)
o = x * 10
o.retain_grad()
o2 = o + 10
o2.retain_grad()
y = torch.rand(10)

In [63]:
o._version  # the version counter is initialized to zero

0

In [64]:
o.add_(-1)  # dirty modify, increase the version counter
o._version

1

In [65]:
z = x + y  # it does not modify x in place
x._version

0

In [66]:
x = x + x  # x is a new tensor
x._version

0

In [67]:
# So... let's break the autodifferentiation with in-place operations

try:
  x = torch.ones(5, requires_grad=True)
  x2 = (x + 1).sqrt()
  z = (x2 - 10)
  x2[0] = -1
  z.sum().backward()
except Exception as e:
  print(e)

one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [5]], which is output 0 of SqrtBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).


References:

- [PyTorch docs](https://pytorch.org/docs/stable/index.html)
- [Autograd tutorial](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
- [Autograd mechanics](https://pytorch.org/docs/stable/notes/autograd.html)
- [Extending PyTorch](https://pytorch.org/docs/stable/notes/extending.html)
- Nice [blogpost](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/) 
- Nice [blogpost](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95) number  two


## JAX: functional differentiation

The approach used by PyTorch or Tensorflow is essentially the same, apart from small differences in how they handle the computational graph. However, it *is not the only possible*.

We will not introduce [**Jax**](https://jax.readthedocs.io/en/latest/) in these notebooks, but it deserves a mention of honor because it employs a fundamental different technique to perform autodiff:

![](https://sjmielke.com/images/blog/jax-purify/banner.png)

# The `torch.nn` package



PyTorch provides the elegantly designed modules and classes 
[`torch.nn`](https://pytorch.org/docs/stable/nn.html), 
[`torch.optim`](https://pytorch.org/docs/stable/optim.html), 
[`Dataset`](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset), 
and [`DataLoader`](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader)
to help you create and train neural networks.
You already saw how to use `torch.optim`, `Dataset` and `DataLoader`. In this section we will review all these classes together with the new `torch.nn` packages to understand how they work together to simplify our life.

In order to fully utilize their power and customize them for your problem, you need to really understand exactly what they're doing. 
To develop this understanding, we will first train basic neural net on the MNIST data set without using any features from these models: we will initially only use the most basic PyTorch tensor functionality. 
Then, we will incrementally add one feature from ``torch.nn``, ``torch.optim``, ``Dataset``, or ``DataLoader`` at a time, showing exactly what each piece does, and how it works to make the code either more concise, or more flexible.

---

Our goal is to reach an elegant, general structure suitable for most problems and models with minors tweaks:

```python
# load data
# instantiate model
# instantiate optimizer

# for each epoch
  # train the model on the training set
  # evaluate the model on one or more evaluation sets
  # log metrics (e.g. log, accuracy)
```


### Classifying *all* handwritten digits


We will use newly the classic [`MNIST`](http://deeplearning.net/data/mnist/)
dataset trying to predict the handwritten digits. 

This time we will predict all digits, so it is a multinomial logistic regression. We will encode the digit information in a *one-hot-encoding fashion*, so:

$$0 = (1,0,0,0,0,0,0,0,0,0) $$
$$1 = (0,1,0,0,0,0,0,0,0,0) $$
$$...$$
$$9 = (0,0,0,0,0,0,0,0,0,1) $$

So the output of our model now is a 10-dimensional vector, we want to interpret it as a probability distribution, so summing to $1$, like:

$$(0,0,0.98,0.01,0,0,0,0,0.01,0) $$
That we will interpret as "*the model says that this input is 98\% a $2$, 1% a $3$ and 1% an $8$*".

In the 1-dimensional logistic regression the single neuron in output was esily interpretable as a probability once passed through the sigmoid $\sigma()$, since the sigmoid output is between $0$ and $1$. So an output of $0.17$ of the model in the last notebook was interpreted as "*the model says this input is 17% a $1$ and 83% a $7$*".

Which is the analogous of the sigmoid in more than one dimension? It's the softmax! defined as:

$$softmax(\mathbf{x}) = \{\frac{exp(x_0)}{\sum_{j}^{ }exp(x_j))}, \frac{exp(x_1)}{\sum_{j}^{ }exp(x_j))}, ... , \frac{exp(x_9)}{\sum_{j}^{ }exp(x_j))}\}$$

As loss, we will use the negative loglikelihood, i.e. the negative logarithm of the ground truth class.

![nll](https://ljvmiranda921.github.io/assets/png/cs231n-ann/neg_log_demo.png)

What we call cross-entropy loss is just the combination of the softmax and the negative log-likelihood, tipically reduced by a sum or a mean, as shown in the image (sum in this case).




### MNIST data setup

For the sake of this example, *we will avoid the use of the ready-made MNIST dataset in the `torchvision` package* but we will manually build it, to understand how to adapt the concepts on other datasets not available in `torchvision`.

We will use [`pathlib`](https://docs.python.org/3/library/pathlib.html)
for dealing with paths, and will download the dataset using [`requests`](http://docs.python-requests.org/en/master/).

We will only import modules when we use them, so you can see exactly what's being used at each point.



This MNIST dataset is in numpy array format.



In [68]:
!wget https://s3.amazonaws.com/img-datasets/mnist.npz

def load_data_impl():
    # file retrieved by:
    #   wget https://s3.amazonaws.com/img-datasets/mnist.npz -O code/dlgo/nn/mnist.npz
    # code based on:
    #   site-packages/keras/datasets/mnist.py
    path = 'mnist.npz'
    f = np.load(path)
    x_train, y_train = f['x_train'].reshape(-1, 784), f['y_train']
    x_test, y_test = f['x_test'].reshape(-1, 784), f['y_test']
    f.close()
    return (x_train.astype(np.float32), y_train), (x_test.astype(np.float32), y_test)

--2023-03-28 09:23:50--  https://s3.amazonaws.com/img-datasets/mnist.npz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.49.248, 52.217.171.24, 52.217.168.88, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.49.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz’

mnist.npz           100%[===================>]  10.96M  57.5MB/s    in 0.2s    

2023-03-28 09:23:51 (57.5 MB/s) - ‘mnist.npz’ saved [11490434/11490434]



In [69]:
(x_train, y_train), (x_valid, y_valid) = load_data_impl()

> **EXERCISE**: We almost always want to normalize datasets, i.e. we subtract their mean and scale by their standard deviation. Compute the mean and standard deviation of the MNIST dataset and then normalize it.

In [70]:
# Normalization with values pre-computed

x_train = (x_train / 255 - 0.13) / 0.3  # data normalization
x_valid = (x_valid / 255 - 0.13) / 0.3

Each image is `28 x 28`, and is being stored as a flattened row of length
`784 (=28x28)`. Let's take a look at one; we need to reshape it to 2d
first.



In [71]:
import plotly.express as px
import numpy as np

print(x_train.shape)
px.imshow(x_train[0].reshape((28, 28)), color_continuous_scale='gray')

(60000, 784)


PyTorch uses ``torch.tensor``, rather than numpy arrays, so we need to
convert our data:

In [72]:
import torch

x_train, y_train, x_valid, y_valid = map(
  torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
y_train = y_train.long()  # we will use targets as indices and pytorch wants int64 as indices
y_valid = y_valid.long()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333],
        [-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333],
        [-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333],
        ...,
        [-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333],
        [-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333],
        [-0.4333, -0.4333, -0.4333,  ..., -0.4333, -0.4333, -0.4333]]) tensor([5, 0, 4,  ..., 5, 6, 8])
torch.Size([60000, 784])
tensor(0) tensor(9)


### Neural net from scratch (no torch.nn)

As in the last notebook, let's first create a model using nothing but PyTorch tensor operations. 

For the weights, we set `requires_grad` **after** the initialization, since we don't want the initialization function to be included in the gradient computation. (remember that a trailling `_` in PyTorch signifies that the operation is performed in-place.)

We are initializing the weights with
[Xavier initialisation](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), i.e. by multiplying with $\frac{1}{\sqrt{n}}$


In [73]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)  # Xavier initialisation
weights.requires_grad_()                         # Start to track the weights     
bias = torch.zeros(10, requires_grad=True)       # Initialize the bias 

So let's just write a plain matrix multiplication and broadcasted addition to create a simple linear model. We also need an activation function, rather than using the $\text{softmax}$, we will use the $\text{log(softmax)}$ to simplify the implementation of the negative-log-likelihood (we have already done the log here). So we'll write `log_softmax` and use it. Remember: although PyTorch provides lots of pre-written loss functions, activation functions, and so forth, you can easily write your own.

In [74]:
def log_softmax(x):
  return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
  return log_softmax(xb @ weights + bias)

We will call our function on one batch of data (in this case, 64 images). This is one forward pass. Note that our predictions won't be any better than random at this stage, since we start with random weights.

In [75]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # log predictions
preds[0], preds.shape
print(torch.exp(preds[0]), preds.shape)

tensor([0.3714, 0.1170, 0.0343, 0.0124, 0.1040, 0.0534, 0.0446, 0.0895, 0.0412,
        0.1320], grad_fn=<ExpBackward0>) torch.Size([64, 10])


> **EXERCISE**: Look at the picture at the beginning of the section and implement the negative-log-likelihood loss function in standard python. Remember, we have already done the log (the model outputs a log-softmax rather than a softmax).
>
> 1 ⭐: implement it using a `for` loop.
>
> 2 ⭐: implement it without loops. 
>*Hint:* You may find useful the `torch.gather` function


In [76]:
def nll(input, target):
    # your code here
    return loss

In [77]:
# @title Solution 👀

def nll(input, target):
    loss = -(input.gather(1, target.view(-1,1))).mean()
    return loss

loss_func = nll

Let's check our loss with our random model, so we can see if we improve
after a backprop pass later.


In [79]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

tensor(2.7133, grad_fn=<NegBackward0>)


Let's also implement a function to calculate the accuracy of our model. For each prediction, if the index with the largest value matches the target value, then the prediction was correct:

In [80]:
def accuracy(out, yb):
  preds = torch.argmax(out, dim=1)
  return (preds == yb).float().mean()

Let's check the accuracy of our random model, so we can see if our
accuracy improves as our loss improves:

In [81]:
print(accuracy(preds, yb))

tensor(0.1094)


We can now run a training loop.  For each iteration, we will:

- select a mini-batch of data (of size ``bs``)
- use the model to make predictions
- calculate the loss
- ``loss.backward()`` computes the gradient of the loss w.r.t. to `weights` and `bias`.

We now use these gradients to update the weights and bias.  We do this
within the ``torch.no_grad()`` context manager, because we do not want these
actions to be recorded for our next calculation of the gradient. 

We then set the gradients to zero, so that we are ready for the next loop.
Otherwise, our gradients would record a running tally of all the operations
that had happened 
(i.e. ``loss.backward()`` *adds* the gradients to whatever is already stored, rather than replacing them).

---
**Debugging: USE THE DEBUGGER!**

You can use the normal python debuggers (e.g. the one integrated in [PyCharm](https://www.jetbrains.com/pycharm/), pro license is free for students) to step through PyTorch code, allowing you to check the various variable values at each step.

On colab, **unfortunately**, you can only use the built-in debugger available in `IPython.core.debugger`. Uncomment ``set_trace()`` below to try it out, [here](https://nblock.org/2011/11/15/pdb-cheatsheet/) you can find a cheatsheet of the pdb commands.




In [83]:
from IPython.core.debugger import set_trace

lr = 0.05  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
  for i in range((n - 1) // bs + 1):
    start_i = i * bs
    # set_trace()

    end_i = start_i + bs
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]
    pred = model(xb)
    loss = loss_func(pred, yb)

    loss.backward()
    with torch.no_grad():
      weights -= weights.grad * lr

      bias -= bias.grad * lr
      weights.grad.zero_()
      bias.grad.zero_()

That's it: we've created and trained a minimal neural network (in this case, a logistic regression, since we have no hidden layers) entirely from scratch!

Let's check the loss and accuracy and compare those to what we got
earlier. We expect that the loss will have decreased and accuracy to
have increased, and they have.



In [84]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0475, grad_fn=<NegBackward0>) tensor(1.)


**EXERCISE**
>
> Why the accuracy is able to reach one, i.e. perfect score? 
> Is the model just extremely good?

### Refactor: use `torch.nn.functional`


We will now refactor our code, so that it does the same thing as before, only
we'll start taking advantage of PyTorch's ``nn`` classes to make it more concise
and flexible. At each step from here, we should be making our code one or more
of: shorter, more understandable, and/or more flexible.

The first and easiest step is to make our code shorter by replacing our
hand-written activation and loss functions with those from ``torch.nn.functional``
(which is generally imported into the namespace ``F`` by convention). This module
contains all the functions in the ``torch.nn`` library (whereas other parts of the
library contain classes). As well as a wide range of loss and activation
functions, you'll also find here some convenient functions for creating neural
nets, such as pooling functions. (There are also functions for doing convolutions,
linear layers, etc, but as we'll see, these are usually better handled using
other parts of the library.)

If you're using negative log likelihood loss and log softmax activation,
then Pytorch provides a single function ``F.cross_entropy`` that combines
the two, and is numerically more stable. So we can even remove the activation function from our model.


In [85]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
  return xb @ weights + bias

Note that we no longer call ``log_softmax`` in the ``model`` function. Let's
confirm that our loss and accuracy are the same as before:

In [86]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # The grad_fn changed!

tensor(0.0475, grad_fn=<NllLossBackward0>) tensor(1.)


### Refactor: use `nn.Module`

Next up, we'll use ``nn.Module`` and ``nn.Parameter``, for a clearer and more
concise training loop. We subclass ``nn.Module`` (which itself is a class and
able to keep track of state).  In this case, we want to create a class that
holds our weights, bias, and method for the forward step.  ``nn.Module`` has a
number of attributes and methods (such as ``.parameters()`` and ``.zero_grad()``)
which we will be using.

---
Note that ``nn.Module`` (uppercase M) is a PyTorch specific concept, and is a
   class we'll be using a lot. ``nn.Module`` is not to be confused with the Python
   concept of a (lowercase ``m``) [`module`](https://docs.python.org/3/tutorial/modules.html)
   which is a file of Python code that can be imported.
   



In [87]:
from torch import nn

class Mnist_Logistic(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
    self.bias = nn.Parameter(torch.zeros(10))

  def forward(self, xb):
    return xb @ self.weights + self.bias

Since we're now using an object instead of just using a function, we
first have to instantiate our model:



In [88]:
model = Mnist_Logistic()

Now we can calculate the loss in the same way as before. Note that
``nn.Module`` objects are used as if they are functions (i.e they are
*callable*), but behind the scenes Pytorch will call our ``forward``
method automatically. 

> The `__call__` method of the Modules, internally calls the `forward` method and *does other stuff* (e.g. registers some hooks, you can check the implementation [here](https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module)). Thus, you should always call the forward with `model(inputs)` and never directly the forward `model.forward(inputs)`.

In [89]:
print(loss_func(model(xb), yb))

tensor(2.9136, grad_fn=<NllLossBackward0>)


Previously for our training loop we had to update the values for each parameter
by name, and manually zero out the grads for each parameter separately, like this:

```python
  with torch.no_grad():
      weights -= weights.grad * lr
      bias -= bias.grad * lr
      weights.grad.zero_()
      bias.grad.zero_()
```

Now we can take advantage of `model.parameters()` and `model.zero_grad()` (which are both defined by PyTorch for ``nn.Module``) to make those steps more concise and less prone to the error of forgetting some of our parameters, particularly if we had a more complicated model:

```python
  with torch.no_grad():
      for p in model.parameters(): p -= p.grad * lr
      model.zero_grad()
```

We'll wrap our little training loop in a ``fit`` function so we can run it
again later.



In [90]:
def fit():
  for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
      start_i = i * bs
      end_i = start_i + bs
      xb = x_train[start_i:end_i]
      yb = y_train[start_i:end_i]
      pred = model(xb)
      loss = loss_func(pred, yb)

      loss.backward()
      with torch.no_grad():
        for p in model.parameters():
          p -= p.grad * lr
        model.zero_grad()

fit()

Let's double-check that our loss has gone down:

In [91]:
print(loss_func(model(xb), yb))

tensor(0.0528, grad_fn=<NllLossBackward0>)


### Refactor: use `nn.Linear`



We continue to refactor our code.  Instead of manually defining and
initializing ``self.weights`` and ``self.bias``, and calculating ``xb  @
self.weights + self.bias``, we will instead use the Pytorch class
[`nn.Linear`](https://pytorch.org/docs/stable/nn.html#linear-layers) for a
linear layer, which does all that for us. 

Pytorch has many types of
predefined layers that can greatly simplify our code, and often makes it
faster too.

In [92]:
class Mnist_Logistic(nn.Module):
  def __init__(self):
    super().__init__()
    self.lin = nn.Linear(784, 10)

  def forward(self, xb):
    return self.lin(xb)

We instantiate our model and calculate the loss in the same way as before:

In [93]:
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

tensor(2.4473, grad_fn=<NllLossBackward0>)


We are still able to use our same ``fit`` method as before:

In [94]:
fit()

print(loss_func(model(xb), yb))

tensor(0.0526, grad_fn=<NllLossBackward0>)


### Refactor: use `torch.optim`

We already explored the `torch.optim` package in the *Logistic Regression and Optimization* notebook: obviously we can use optimizers to train our models!

We can use the ``step`` method from our optimizer to take a forward step, instead
of manually updating each parameter.

This will let us replace our previous custom optimization step:

```python
with torch.no_grad():
  for p in model.parameters(): p -= p.grad * lr
  model.zero_grad()
```

and instead use just:
```python
opt.step()
opt.zero_grad()
```
to use some fancy optimizer.

(``optim.zero_grad()`` resets the gradient of the tensors being optimized to 0 and we need to call it before computing the gradient for the next minibatch.)

In [95]:
from torch import optim

We'll define a little function to create our model and optimizer so we
can reuse it in the future:

In [96]:
def get_model():
  model = Mnist_Logistic()
  return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
  for i in range((n - 1) // bs + 1):
    start_i = i * bs
    end_i = start_i + bs
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]
    pred = model(xb)
    loss = loss_func(pred, yb)

    loss.backward()
    opt.step()
    opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(2.4078, grad_fn=<NllLossBackward0>)
tensor(0.0533, grad_fn=<NllLossBackward0>)


### Refactor: use `Dataset`


We already explored the PyTorch abstract Dataset class in the *Linear models and Pytorch Datasets* notebook.  


PyTorch's [`TensorDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) 
is a `Dataset` that wraps tensors. This also gives us a way to easily iterate, index, and slice along the first dimension of a tensor. This will make it easier to access both the independent and dependent variables in the same line as we train.

In [97]:
from torch.utils.data import TensorDataset

Both ``x_train`` and ``y_train`` can be combined in a single ``TensorDataset``,
which will be easier to iterate over and slice:



In [98]:
train_ds = TensorDataset(x_train, y_train)

Previously, we had to iterate through minibatches of x and y values separately:

```python
    xb = x_train[i*bs : i*bs+bs]
    yb = y_train[i*bs : i*bs+bs]
```

Now, we can do these two steps together:

```python
    xb, yb = train_ds[i*bs : i*bs+bs]
```



In [99]:
model, opt = get_model()

for epoch in range(epochs):
  for i in range((n - 1) // bs + 1):
    xb, yb = train_ds[i * bs: i * bs + bs]
    pred = model(xb)
    loss = loss_func(pred, yb)

    loss.backward()
    opt.step()
    opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0525, grad_fn=<NllLossBackward0>)


### Refactor: using `DataLoader`

We already explored the PyTorch abstract `DataLoader` class in the *Linear models and Pytorch Datasets* notebook.  

Remember that among the other things Pytorch's ``DataLoader`` is responsible for managing batches and shuffling:
the DataLoader gives us each minibatch automatically.


In [100]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

Previously, our loop iterated over batches (xb, yb) like this:

```python
for i in range((n-1)//bs + 1):
  xb,yb = train_ds[i*bs : i*bs+bs]
  pred = model(xb)
```

Now, our loop is much cleaner, as (xb, yb) are loaded automatically from the data loader:

```python
for xb, yb in train_dl:
  pred = model(xb)
```


In [101]:
model, opt = get_model()

for epoch in range(epochs):
  for xb, yb in train_dl:
    pred = model(xb)
    loss = loss_func(pred, yb)

    loss.backward()
    opt.step()
    opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.3224, grad_fn=<NllLossBackward0>)


Thanks to Pytorch's ``nn.Module``, ``nn.Parameter``, ``Dataset``, and ``DataLoader``,
our training loop is now dramatically smaller and easier to understand. Let's
now try to add the basic features necessary to create effecive models in practice.


### Add: validation set

In reality, you **always** should also have
a [`validation set`](https://www.fast.ai/2017/11/13/validation-sets/), in order to identify if you are overfitting.

Shuffling the training data is important to prevent correlation between batches and overfitting. On the other hand, the validation loss will be identical whether we shuffle the validation set or not. Since shuffling takes extra time and makes qualitative comparisons more difficult, it makes no sense to shuffle the validation data.

We'll use a batch size for the validation set that is twice as large as
that for the training set. This is because the validation set does not
need backpropagation and thus takes less memory (it doesn't need to
store the gradients). We take advantage of this to use a larger batch
size and compute the loss more quickly.




In [102]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

We will calculate and print the validation loss at the end of each epoch.

(Note that we always call ``model.train()`` before training, and ``model.eval()``
before inference, because these modal states are used by layers such as ``nn.BatchNorm2d``
and ``nn.Dropout`` to ensure appropriate behaviour for these different phases.)



In [103]:
model, opt = get_model()

for epoch in range(epochs):
  model.train()
  for xb, yb in train_dl:
    pred = model(xb)
    loss = loss_func(pred, yb)

    loss.backward()
    opt.step()
    opt.zero_grad()

  model.eval()
  with torch.no_grad():
    valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

  print(epoch, valid_loss / len(valid_dl))

0 tensor(0.2940)
1 tensor(0.2846)


### Add: fit() and get_data()

We'll now do a little refactoring of our own. Since we go through a similar
process twice of calculating the loss for both the training set and the
validation set, let's make that into its own function, ``step``, which
computes the loss for one batch and possibly one optimization step.

We pass an optimizer in for the training set, and use it to perform
backprop.  For the validation set, we don't pass an optimizer, so the
method doesn't perform backprop.

In [104]:
def step(model, loss_func, xb, yb, opt=None):
  loss = loss_func(model(xb), yb)

  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()

  return loss.item(), len(xb)

``fit`` runs the necessary operations to train our model and compute the
training and validation losses for each epoch:

In [105]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
  for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
      step(model, loss_func, xb, yb, opt)

    model.eval()
    with torch.no_grad():
      losses, nums = zip(
        *[step(model, loss_func, xb, yb) for xb, yb in valid_dl]
      )
    val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

    print(epoch, val_loss)

``get_data`` returns dataloaders for the training and validation sets.

In [106]:
def get_data(train_ds, valid_ds, bs):
  return (
    DataLoader(train_ds, batch_size=bs, shuffle=True),
    DataLoader(valid_ds, batch_size=bs * 2),
  )

Now, our whole process of obtaining the data loaders and fitting the
model can be run in 3 lines of code:

In [107]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.30677464908361435
1 0.2888036703824997


You can use these basic 3 lines of code to train a wide variety of models.
Let's see if we can use them to train a convolutional neural network (CNN, you will learn about them in the next lessons)!


### Switch to CNN

We are now going to build our neural network with three convolutional layers.
Because none of the functions in the previous section assume anything about
the model form, we'll be able to use them to train a CNN without any modification.

We will use Pytorch's predefined [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) class
as our convolutional layer. We define a CNN with 3 convolutional layers.
Each convolution is followed by a ReLU.  At the end, we perform an
average pooling. 



In [108]:
class Mnist_CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
    self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
    self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

  def forward(self, xb):
    xb = xb.view(-1, 1, 28, 28)
    xb = F.relu(self.conv1(xb))
    xb = F.relu(self.conv2(xb))
    xb = F.relu(self.conv3(xb))
    xb = F.avg_pool2d(xb, 4)
    return xb.view(-1, xb.size(1))

lr = 0.1

In [109]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.2820812686085701
1 0.2111318999826908


### Use: nn.Sequential

``torch.nn`` has another handy class we can use to simplify our code [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential).

A ``Sequential`` object runs each of the modules contained within it, in a
sequential manner. This is a simpler way of writing our neural network.

To take advantage of this, we need to be able to easily define a
**custom layer** from a given function.  For instance, PyTorch doesn't
have a `view` layer, and we need to create one for our network. ``Lambda``
will create a layer that we can then use when defining a network with
``Sequential``.

In [110]:
class Lambda(nn.Module):
  def __init__(self, func):
    super().__init__()
    self.func = func

  def forward(self, x):
    return self.func(x)


def preprocess(x):
  return x.view(-1, 1, 28, 28)

The model created with ``Sequential`` is simply:

In [111]:
model = nn.Sequential(
  Lambda(preprocess),
  nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.AvgPool2d(4),
  Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.2440386187851429
1 0.19654418625831604


### Use: DataLoader wrappers

Our CNN is fairly concise, but it only works with MNIST, because:
 - It assumes the input is a 28\*28 long vector
 - It assumes that the final CNN grid size is 4\*4 (since that's the average
pooling kernel size we used)

Let's get rid of these two assumptions, so our model works with any 2d
single channel image. First, we can remove the initial Lambda layer
moving the data preprocessing into a python generator:



In [112]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y


class WrappedDataLoader:
  def __init__(self, dl, func):
    self.dl = dl
    self.func = func

  def __len__(self):
    return len(self.dl)

  def __iter__(self):
    batches = iter(self.dl)
    for b in batches:
      yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Next, we can replace ``nn.AvgPool2d`` with ``nn.AdaptiveAvgPool2d``, which
allows us to define the size of the *output* tensor we want, rather than
the *input* tensor we have. As a result, our model will work with any
size input.

In [113]:
model = nn.Sequential(
  nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
  nn.ReLU(),
  nn.AdaptiveAvgPool2d(1),
  Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Let's try it out:

In [114]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.23026074725687504
1 0.18931632181406022


### Use: your GPU

If you're lucky enough to have access to a CUDA-capable GPU, you can
use it to speed up your code. First check that your GPU is working in
Pytorch:



In [115]:
print(torch.cuda.is_available())

True


And then create a device object for it:



In [116]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Let's update ``preprocess`` to move batches to the GPU *(usually you should move to GPU the whole batch returned by the dataloader, since it is faster --one mover vs `batch_size` moves-- and the multiple workers of a dataloader together with cuda tensors may cause problems; but let's keep thing simple here)*:



In [117]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Finally, we can move our model to the GPU.



In [118]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

You should find it runs faster now:



In [119]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.14725875496268273
1 0.14849480872750281


### Closing thoughts


We now have a general data pipeline and training loop which you can use for
training many types of models using Pytorch. 

Of course, there are many things you'll want to add, such as data augmentation, hyperparameter tuning, monitoring training, transfer learning, and so forth.

Let's summarize what we've seen:

 - **torch.nn**

   + ``Module``: creates a callable which behaves like a function, but can also
     contain state (such as neural net layer weights). It knows what ``Parameter`` (s) it
     contains and can zero all their gradients, loop through them for weight updates, etc.
   + ``Parameter``: a wrapper for a tensor that tells a ``Module`` that it has weights
     that need updating during backprop. Only tensors with the `requires_grad` attribute set are updated
   + ``functional``: a module (usually imported into the ``F`` namespace by convention)
     which contains activation functions, loss functions, etc, as well as non-stateful
     versions of layers such as convolutional and linear layers.
 - ``torch.optim``: Contains optimizers such as ``SGD``, which update the weights
   of ``Parameter`` during the backward step
 - ``Dataset``: An abstract interface of objects with a ``__len__`` and a ``__getitem__``,
   including classes provided with Pytorch such as ``TensorDataset``
 - ``DataLoader``: Takes any ``Dataset`` and creates an iterator which returns batches of data.

---

Tutorial on `torch.nn` adapted from this [tutorial](https://pytorch.org/tutorials/beginner/nn_tutorial.html).


## Playground: MLPs expressivity



Which type of function does a MLP learn?

To help our intuition let's try to visualize the *functions that a deep mlp learns*, while changing some hyperparameters.

In order to do that, we will use this simple parametric model, where the dimension of the intermediate tensors (the **hidden dimension**), the **number of layers** and the **type of activation** are parametric:

In [120]:
class MLP2D(nn.Module):
  def __init__(self, 
               num_layers: int, 
               hidden_dim: int, 
               activation: Callable[[torch.Tensor], torch.Tensor]
               ) -> None:
    super().__init__()

    self.first_layer = nn.Linear(in_features=2, 
                                 out_features=hidden_dim)

    self.layers = nn.ModuleList()  # A list of modules: automatically exposes nested parameters to optimize. 
                                   # Parameters contained in a normal python list are not returned by model.parameters()
    for i in range(num_layers):
      self.layers.append(
          nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
      )
    self.activation = activation

    self.last_layer = nn.Linear(in_features=hidden_dim, 
                                out_features=1)


  def forward(self, meshgrid: torch.Tensor) -> torch.Tensor:
    """
    Applies transformations to each (x, y) independently 

    :param meshgrid: tensor of dimensions [..., 2], where ... means any number of dims
    """
    out = meshgrid

    out = self.first_layer(out)  # First linear layer, transforms the hidden dimensions from 2 (the coordinates) to `hidden_dim`
    for layer in self.layers:    # Apply `k` (linear, activation) layer
      out = layer(out)
      out = self.activation(out)
    out = self.last_layer(out)   # Last linear layer to bring the `hiddem_dim` features back to the 2 coordinates x, y

    return out.squeeze(-1)


Let's try this model to understand how it works:

In [121]:
model = MLP2D(num_layers=3, 
              hidden_dim=10, 
              activation=torch.nn.functional.relu)

In [122]:
model.parameters  # Explore the parameters, i.e. the trainable tensors that require a grad in this model

<bound method Module.parameters of MLP2D(
  (first_layer): Linear(in_features=2, out_features=10, bias=True)
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
  (last_layer): Linear(in_features=10, out_features=1, bias=True)
)>

Let's try to execute this model on some input:

In [123]:
model(torch.as_tensor([1., 2.]))

tensor(0.1476, grad_fn=<SqueezeBackward1>)

In [124]:
model(torch.as_tensor([[1., 2.]]))

tensor([0.1476], grad_fn=<SqueezeBackward1>)

In [125]:
model(torch.as_tensor([[1., 2.], [1., 2.]]))

tensor([0.1476, 0.1476], grad_fn=<SqueezeBackward1>)

In [126]:
model(torch.rand(2, 3, 2, 2))

tensor([[[0.1559, 0.1552],
         [0.1521, 0.1504],
         [0.1390, 0.1546]],

        [[0.1501, 0.1576],
         [0.1477, 0.1445],
         [0.1431, 0.1519]]], grad_fn=<SqueezeBackward1>)

Now we are going to choose a base function and sample some points over that function, to create our dataset. Our model will try to reconstruct the function (unknown to him) from these points!

We can use a `Dataset` to keep the code clean and organize the sampled points. 
Keep in mind, for each point the model will receive the $(x, y)$ coordinates and try to predict the $z = f(x, y)$ coordinate.

In [127]:
from typing import Dict

class PointsDataset(torch.utils.data.Dataset):
  def __init__(self, x: torch.Tensor, y_true: torch.Tensor) -> None:
    super().__init__()

    self.x = x
    self.y_true = y_true

  def __len__(self) -> int:
    return self.y_true.shape[0]

  def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
    return {                    # For the idx-th sample
        'x': self.x[idx, ...],  # the "x" are the (x, y) coordinates of the idx-th point
        'y': self.y_true[idx]   # the "y" is  the (z) coordinate of the idx-th point
    }

In [129]:
# @title Dataset creation { run: "auto" }

fn_names = {
    'peaks': peaks,
    'rastrigin': rastrigin,
    'rosenbrock': rosenbrock,
    'simple_fn': simple_fn,
    'simple_fn2': simple_fn2
}

base_fn = 'peaks' #@param ["simple_fn", "simple_fn2", "peaks",  "rastrigin", "rosenbrock"]
n_rand = 80 #@param {type:"slider", min:0, max:200, step:1}
plot_lim = 3  #@param {type:"slider", min:0, max:50, step:1}
plot_height = 700 #@param {type:"slider", min:0, max:1500, step:50}

lim = plot_lim
fn = fn_names[base_fn]

x_random = torch.rand(n_rand) * lim * 2 - lim
y_random = torch.rand(n_rand) * lim * 2 - lim

xy_data = torch.stack((x_random, y_random), dim=-1)
xy_groundtruth = fn(xy_data)


points_dl =  torch.utils.data.DataLoader(
    PointsDataset(x=xy_data, y_true=xy_groundtruth),
    batch_size=16,
    shuffle=True    
)

points = torch.cat((xy_data, xy_groundtruth[..., None]), dim=-1)


plot_points_over_landscape(fn, points, lim=lim, height=plot_height)

In [131]:
# @title Learned function from sampled points { run: "auto" }

activation_names = {
    'relu': torch.relu,
    'leaky_relu': torch.nn.functional.leaky_relu,
    'elu': torch.nn.functional.elu,
    'sigmoid': torch.sigmoid,
    'tanh': torch.tanh,
}

num_layers = 2 #@param {type:"slider", min:0, max:20, step:1}

activation_fn = 'relu' #@param ["relu", "leaky_relu", "elu", "sigmoid",  "tanh"]
activation_fn = activation_names[activation_fn]

hidden_dim = 16 #@param {type:"slider", min:0, max:512, step:1}

num_epochs = 610 #@param {type:"slider", min:0, max:1000, step:10}
learning_rate = 0.001 #@param {type:"number"}

plot_lim = 3  #@param {type:"slider", min:0, max:50, step:1}
plot_height = 900 #@param {type:"slider", min:0, max:1500, step:50}

def energy(y_pred, y_true):
  return torch.nn.functional.mse_loss(y_pred, y_true)


model = MLP2D(num_layers=num_layers, activation=activation_fn, hidden_dim=hidden_dim)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

for i in tqdm(range(num_epochs), desc="epoch"):
    for batch in points_dl:
      x = batch['x'].to(device)
      y = batch['y'].to(device)
      y_pred = model(x)

      loss = energy(y_pred, y)

      loss.backward()
      opt.step()
      opt.zero_grad()
    

plot_points_over_landscape(model.cpu(), points.cpu(), lim=lim, height=plot_height).show()

epoch:   0%|          | 0/610 [00:00<?, ?it/s]

!!!

If you have problems visualizing the previous animation in Google Colab, please explore it in this streamlit app:

-  https://lucmos-demo-nn-expressivity-ui-plhjjk.streamlit.app/